### Project COEUR

In [41]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, classification_report

### Charge le dataset coeur

In [42]:
# Charge le dataset.
data = pd.read_excel("dataset/coeur.xlsx")

In [43]:
# Copie data dans df.
df = data.copy()

In [44]:
# Affiche les informations relatives à df.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   AGE          918 non-null    int64  
 1   SEXE         918 non-null    object 
 2   TDT          918 non-null    object 
 3   PAR          918 non-null    int64  
 4   CHOLESTEROL  918 non-null    int64  
 5   GAJ          918 non-null    int64  
 6   ECG          918 non-null    object 
 7   FCMAX        918 non-null    int64  
 8   ANGINE       918 non-null    object 
 9   DEPRESSION   918 non-null    float64
 10  PENTE        918 non-null    object 
 11  CŒUR         918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [45]:
# Affiche les prémières lignes de df.
df.head()

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,40,homme,AA,140,289,0,Normal,172,Non,0.0,Ascendant,0
1,49,femme,DNA,160,180,0,Normal,156,Non,1.0,Plat,1
2,37,homme,AA,130,283,0,ST,98,Non,0.0,Ascendant,0
3,48,femme,ASY,138,214,0,Normal,108,Oui,1.5,Plat,1
4,54,homme,DNA,150,195,0,Normal,122,Non,0.0,Ascendant,0


### Vérifie les doublons

In [46]:
# Compte le nombre de lignes dupliquées (ici il n'y en a pas).
df.duplicated().sum()

0

### Vérifie les données manquantes

In [47]:
# Compte le nombre de données manquantes par colonne (ici il n'y en a pas).
df.isna().sum()

AGE            0
SEXE           0
TDT            0
PAR            0
CHOLESTEROL    0
GAJ            0
ECG            0
FCMAX          0
ANGINE         0
DEPRESSION     0
PENTE          0
CŒUR           0
dtype: int64

### Vérifie les constantes

In [48]:
# Ici il n'y a pas de constantes, car chaque variable prend au moins deux valeurs différentes.
df.nunique()

AGE             50
SEXE             2
TDT              4
PAR             67
CHOLESTEROL    222
GAJ              2
ECG              3
FCMAX          119
ANGINE           2
DEPRESSION      53
PENTE            3
CŒUR             2
dtype: int64

### Normalise les variables quantitatives

In [49]:
# Normalise les colonnes de type number.
for column in df.drop("CŒUR", axis=1).select_dtypes(np.number).columns:
    df[column] = df[column] / df[column].max()

In [50]:
# Vérifie le résultat.
df.head()

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,0.519481,homme,AA,0.70,0.479270,0.0,Normal,0.851485,Non,0.000000,Ascendant,0
1,0.636364,femme,DNA,0.80,0.298507,0.0,Normal,0.772277,Non,0.161290,Plat,1
2,0.480519,homme,AA,0.65,0.469320,0.0,ST,0.485149,Non,0.000000,Ascendant,0
3,0.623377,femme,ASY,0.69,0.354892,0.0,Normal,0.534653,Oui,0.241935,Plat,1
4,0.701299,homme,DNA,0.75,0.323383,0.0,Normal,0.603960,Non,0.000000,Ascendant,0


### Encode les variables qualitatives

In [51]:
# Encode les variables qualitatives.
for column in df.drop("CŒUR", axis=1).select_dtypes("object").columns:
        df[column] = df[column].astype("category").cat.codes

In [52]:
# Vérifie le résultat.
df.head()

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,0.519481,1,0,0.70,0.479270,0.0,1,0.851485,0,0.000000,0,0
1,0.636364,0,3,0.80,0.298507,0.0,1,0.772277,0,0.161290,2,1
2,0.480519,1,0,0.65,0.469320,0.0,2,0.485149,0,0.000000,0,0
3,0.623377,0,1,0.69,0.354892,0.0,1,0.534653,1,0.241935,2,1
4,0.701299,1,3,0.75,0.323383,0.0,1,0.603960,0,0.000000,0,0


### Défini Features et Target

In [53]:
# Sépare df: x les variables explicatives et y la variable cible (coeur).
x = df.drop("CŒUR", axis=1)
y = df.CŒUR

In [54]:
# Vérifie le résultat.
x.head()

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE
0,0.519481,1,0,0.70,0.479270,0.0,1,0.851485,0,0.000000,0
1,0.636364,0,3,0.80,0.298507,0.0,1,0.772277,0,0.161290,2
2,0.480519,1,0,0.65,0.469320,0.0,2,0.485149,0,0.000000,0
3,0.623377,0,1,0.69,0.354892,0.0,1,0.534653,1,0.241935,2
4,0.701299,1,3,0.75,0.323383,0.0,1,0.603960,0,0.000000,0


In [55]:
# Vérifie le résultat.
y.head()

0    0
1    1
2    0
3    1
4    0
Name: CŒUR, dtype: int64

### Répartis les données du dataset

In [56]:
# Répartis le jeu de données, 70% pour l'entraînement et 30% pour les tests.
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

In [57]:
# Vérifie le résultat.
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(642, 11) (276, 11) (642,) (276,)


### La régression logistique

In [58]:
# Instancie la classe de régression logistique.
logistic_regression = LogisticRegression(random_state=0)

In [59]:
# Entraîne le modèle de régression logistique.
logistic_regression.fit(x_train, y_train)

LogisticRegression(random_state=0)

In [60]:
# Le score d'entraînement: taux de bonnes prédictions calculé sur la base des données d'entraînement.
logistic_regression.score(x_train, y_train)

0.8660436137071651

### L'arbre de décision

In [61]:
# Instancie la classe d'arbre de décision.
decision_tree = DecisionTreeClassifier(random_state=0)

In [62]:
# Entraîne notre modèle d'arbre de décision.
decision_tree.fit(x_train, y_train)

DecisionTreeClassifier(random_state=0)

In [63]:
# Le score d'entraînement: taux de bonnes prédictions calculé sur la base des données d'entraînement.
decision_tree.score(x_train, y_train)

1.0

### La prédiction

In [64]:
# La prédiction de la régression logistique.
lr_pred = logistic_regression.predict(x_test)

In [65]:
# La prédiction de l'abre de décision.
tree_pred = decision_tree.predict(x_test)

### La matrice de confusion

In [66]:
# La matrice de confusion de la régression logistique.
lr_confusion_matrix = confusion_matrix(y_test, lr_pred)
lr_confusion_matrix

array([[ 91,  22],
       [ 26, 137]], dtype=int64)

In [67]:
# La matrice de confusion de l'arbre de décision.
tree_confusion_matrix = confusion_matrix(y_test, tree_pred)
tree_confusion_matrix

array([[ 90,  23],
       [ 34, 129]], dtype=int64)

### Le taux de bonnes prédictions

In [68]:
# Le taux de bonnes prédictions de la régression logistique.
lr_accuracy = accuracy_score(y_test, lr_pred)
lr_accuracy

# Interprétation: le modèle produit plus 82% de bonnes prédictions.

0.8260869565217391

In [69]:
# Le taux de bonnes prédictions de l'arbre de décision.
tree_accuracy = accuracy_score(y_test, tree_pred)
tree_accuracy

# Interprétation: le modèle produit plus 79% de bonnes prédictions.

0.7934782608695652

### La sensibilité

In [70]:
# La sensibilité de la régression logistique.
lr_recall = recall_score(y_test, lr_pred)
lr_recall

# Interprétation: lorsqu'un individu est malade, le modèle a plus de 84% de chance de le prédire.

0.8404907975460123

In [71]:
# La sensibilité de l'arbre de décision.
tree_recall = recall_score(y_test, tree_pred)
tree_recall

# Interprétation: lorsqu'un individu est malade, le modèle a plus de 79% de chance de le prédire.

0.7914110429447853

### La précision

In [72]:
# La précision de la régression logistique.
lr_precision = precision_score(y_test, lr_pred)
lr_precision

# Interprétation: lorsque le modèle prédit qu'un individu est malade, il y a plus de 86% de chance que cela soit vraie.

0.8616352201257862

In [73]:
# La précision de l'arbre de décision.
tree_precision = precision_score(y_test, tree_pred)
tree_precision

# Interprétation: lorsque le modèle prédit qu'un individu est malade, il y a plus de 84% de chance que cela soit vraie.

0.8486842105263158

### Le rapport de classification

In [74]:
# Le rapport de classification de la régression logistique.
lr_classification_report = classification_report(y_test, lr_pred)
print(lr_classification_report)

              precision    recall  f1-score   support

           0       0.78      0.81      0.79       113
           1       0.86      0.84      0.85       163

    accuracy                           0.83       276
   macro avg       0.82      0.82      0.82       276
weighted avg       0.83      0.83      0.83       276



In [75]:
# Le rapport de classification de l'arbre de décision.
tree_classification_report = classification_report(y_test, tree_pred)
print(tree_classification_report)

              precision    recall  f1-score   support

           0       0.73      0.80      0.76       113
           1       0.85      0.79      0.82       163

    accuracy                           0.79       276
   macro avg       0.79      0.79      0.79       276
weighted avg       0.80      0.79      0.79       276



### Le modèle trival

In [76]:
# Le modèle trival.
df.CŒUR.value_counts() / df.shape[0]

# Interprétation: si nous nous basons sur le hasard pour faire nos prédictions, nous aurons plus de 55% de bonnes prédictions.

1    0.553377
0    0.446623
Name: CŒUR, dtype: float64

### La performance

In [77]:
# La performance de la régression logistique.
logistic_regression.score(x_test, y_test)

0.8260869565217391

In [78]:
# La performance de l'arbre de décision.
decision_tree.score(x_test, y_test)

0.7934782608695652